In [5]:
import requests
import json
import pandas as pd
import feedparser as fp
import newspaper as np

def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories):
    # Liste des nouveaux articles
    new_articles = []

    # Parcourir les flux RSS
    for source_url in flux_rss_a_lire:
        # Déterminer la catégorie du flux
        category = categories[source_url]

        data = fp.parse(source_url)
        
        # Parcourir les articles du flux RSS
        for item in data.entries:
            # Clés renommées pour correspondre à la base de données existante
            article = {
                "title": item.title,
                "date": item.published,  # Renommé de 'published' à 'date'
                "url": item.link,  # Renommé de 'link' à 'url'
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": category,  # Ajouté la catégorie
            }
            
            # Vérifier si l'article existe déjà dans la base de données
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                
                # Mettre à jour les informations de l'article
                article["content"] += "\n\n" + article_obj.text  # Ajouter le contenu téléchargé
                article["author"] = article_obj.authors  # Ajouté la liste des auteurs
                
                # Ajouter l'article à la liste des nouveaux articles
                new_articles.append((article["url"], article))

    # Mettre à jour la base de données existante avec les nouveaux articles
    for url, article in new_articles:
        database[url] = article

    return database


# Lien vers le fichier tvinfo-sources.json
tvinfo_sources_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json"

# Charger la liste des flux RSS depuis le lien tvinfo-sources.json
response_sources = requests.get(tvinfo_sources_url)
sources = json.loads(response_sources.text)

# Liste des flux RSS à parcourir
flux_rss_a_lire = list(sources.values())

# Usage de la fonction avec les catégories
categories = {
    "https://www.francetvinfo.fr/france.rss": "france",
    "https://www.francetvinfo.fr/monde/europe.rss": "europe",  # Corrigé ici
    "https://www.francetvinfo.fr/economie/entreprises.rss": "economie"
}


# Charger la base de données existante depuis le lien francetvinfo.json
francetvinfo_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json"
response_database = requests.get(francetvinfo_url)
database = json.loads(response_database.text)

# Mettre à jour la base de données avec les nouveaux articles
database_mise_a_jour = mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories)

# Transformer la base de données mise à jour en DataFrame
df_database_mise_a_jour = pd.DataFrame(database_mise_a_jour.values())

# Ajouter les URLs comme colonne dans le DataFrame
df_database_mise_a_jour['url'] = list(database_mise_a_jour.keys())

# Afficher les premières lignes pour vérifier
df_database_mise_a_jour





,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2151,"Casino : ""On n'a pas prévu de réduire les effe...","Tue, 27 Feb 2024 10:27:58 +0100",[],economie,"""On n'a pas un plan de restructuration en cour...",https://www.francetvinfo.fr/pictures/OPzfYdeKj...,NaN,https://www.francetvinfo.fr/economie/commerce/...
2152,Fret maritime : des profits en chute libre pou...,"Tue, 27 Feb 2024 07:18:33 +0100",[Emmanuel],economie,L’armateur français annonce de mauvais résulta...,https://www.francetvinfo.fr/pictures/954HVxZI9...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2153,Casino : le tribunal de commerce de Paris vali...,"Mon, 26 Feb 2024 17:30:40 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/JYAMPgoom...,NaN,https://www.francetvinfo.fr/economie/entrepris...
2154,Le tribunal de commerce de Paris valide le pla...,"Mon, 26 Feb 2024 17:21:53 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/wowtVTSX3...,NaN,https://www.francetvinfo.fr/economie/entrepris...


,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2151,"Casino : ""On n'a pas prévu de réduire les effe...","Tue, 27 Feb 2024 10:27:58 +0100",[],economie,"""On n'a pas un plan de restructuration en cour...",https://www.francetvinfo.fr/pictures/OPzfYdeKj...,NaN,https://www.francetvinfo.fr/economie/commerce/...
2152,Fret maritime : des profits en chute libre pou...,"Tue, 27 Feb 2024 07:18:33 +0100",[Emmanuel],economie,L’armateur français annonce de mauvais résulta...,https://www.francetvinfo.fr/pictures/954HVxZI9...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2153,Casino : le tribunal de commerce de Paris vali...,"Mon, 26 Feb 2024 17:30:40 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/JYAMPgoom...,NaN,https://www.francetvinfo.fr/economie/entrepris...
2154,Le tribunal de commerce de Paris valide le pla...,"Mon, 26 Feb 2024 17:21:53 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/wowtVTSX3...,NaN,https://www.francetvinfo.fr/economie/entrepris...


# Mettre à jour la base de données de Git

In [6]:

# Fonction pour récupérer la base de données actuelle depuis GitHub
def telecharger_base_donnees_github(url):
    response = requests.get(url)
    data = response.text.splitlines()  # Séparer chaque ligne du fichier JSON
    df = pd.DataFrame([pd.read_json(line, typ='series') for line in data])
    return df

# Fonction pour concaténer l'ancienne base de données avec les nouveaux articles
def concatener_bases_donnees(df_ancienne, df_nouveaux_articles):
    # Filtrer les nouveaux articles pour ne garder que ceux qui n'existent pas déjà dans l'ancienne base de données
    nouveaux_articles_uniques = df_nouveaux_articles[~df_nouveaux_articles['url'].isin(df_ancienne['url'])]
    # Concaténer les deux DataFrames
    df_mise_a_jour = pd.concat([df_ancienne, nouveaux_articles_uniques], ignore_index=True)
    return df_mise_a_jour

# URL de la base de données sur GitHub
url_github = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'

# Charger la base de données actuelle
df_ancienne = telecharger_base_donnees_github(url_github)

# Concaténer l'ancienne base de données avec les nouveaux articles
df_mise_a_jour = concatener_bases_donnees(df_ancienne, df_database_mise_a_jour)

# Sauvegarder la base de données mise à jour localement
df_mise_a_jour.to_json("Data/francetvinfo_maj.json", orient="records", lines=True, force_ascii=False)
df_mise_a_jour.to_csv("Data/francetvinfo_maj.csv", index=False)


In [7]:
df_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2623,"Casino : ""On n'a pas prévu de réduire les effe...","Tue, 27 Feb 2024 10:27:58 +0100",[],economie,"""On n'a pas un plan de restructuration en cour...",https://www.francetvinfo.fr/pictures/OPzfYdeKj...,NaN,https://www.francetvinfo.fr/economie/commerce/...
2624,Fret maritime : des profits en chute libre pou...,"Tue, 27 Feb 2024 07:18:33 +0100",[Emmanuel],economie,L’armateur français annonce de mauvais résulta...,https://www.francetvinfo.fr/pictures/954HVxZI9...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2625,Casino : le tribunal de commerce de Paris vali...,"Mon, 26 Feb 2024 17:30:40 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/JYAMPgoom...,NaN,https://www.francetvinfo.fr/economie/entrepris...
2626,Le tribunal de commerce de Paris valide le pla...,"Mon, 26 Feb 2024 17:21:53 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/wowtVTSX3...,NaN,https://www.francetvinfo.fr/economie/entrepris...


,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2623,"Casino : ""On n'a pas prévu de réduire les effe...","Tue, 27 Feb 2024 10:27:58 +0100",[],economie,"""On n'a pas un plan de restructuration en cour...",https://www.francetvinfo.fr/pictures/OPzfYdeKj...,NaN,https://www.francetvinfo.fr/economie/commerce/...
2624,Fret maritime : des profits en chute libre pou...,"Tue, 27 Feb 2024 07:18:33 +0100",[Emmanuel],economie,L’armateur français annonce de mauvais résulta...,https://www.francetvinfo.fr/pictures/954HVxZI9...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2625,Casino : le tribunal de commerce de Paris vali...,"Mon, 26 Feb 2024 17:30:40 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/JYAMPgoom...,NaN,https://www.francetvinfo.fr/economie/entrepris...
2626,Le tribunal de commerce de Paris valide le pla...,"Mon, 26 Feb 2024 17:21:53 +0100",[],economie,"Grâce à ce feu vert, Casino va pouvoir changer...",https://www.francetvinfo.fr/pictures/wowtVTSX3...,NaN,https://www.francetvinfo.fr/economie/entrepris...


In [8]:
import pandas as pd
# URL du fichier mise à jour chaque jour JSON sur GitHub
url = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'
# Charger les données dans un DataFrame
francetvinfo_mise_a_jour = pd.read_json(url, lines=True)
# Afficher les premières lignes du DataFrame
francetvinfo_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2564,Exosquelettes : on en est où ?,"Sat, 02 Mar 2024 07:59:25 +0100",[Sarah Lemoine],economie,Après avoir longuement testé des exosquelettes...,https://www.francetvinfo.fr/pictures/jXE5_KiLM...,None,https://www.francetvinfo.fr/replay-radio/c-est...
2565,Sept entreprises sur dix se disent touchées pa...,"Fri, 01 Mar 2024 20:34:44 +0100",[Franceinfo Avec Afp],economie,Les disparités sont nettes entre les secteurs....,https://www.francetvinfo.fr/pictures/f71kf5Ytm...,None,https://www.francetvinfo.fr/economie/entrepris...
2566,"Entreprise : après avoir frôlé la faillite, le...","Fri, 01 Mar 2024 18:57:01 +0100","[Camille Revel, Philippe Guillemot]",economie,"En ce moment, c'est la saison des résultats et...",https://www.francetvinfo.fr/pictures/eafiuZ9tK...,None,https://www.francetvinfo.fr/replay-radio/l-int...
2567,Ehpad : un grand groupe de maisons de retraite...,"Fri, 01 Mar 2024 16:02:50 +0100",[],economie,"Medicharme, un grand groupe de maisons de retr...",https://www.francetvinfo.fr/pictures/9b56oeGQL...,None,https://www.francetvinfo.fr/sante/senior/maltr...


,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2564,Exosquelettes : on en est où ?,"Sat, 02 Mar 2024 07:59:25 +0100",[Sarah Lemoine],economie,Après avoir longuement testé des exosquelettes...,https://www.francetvinfo.fr/pictures/jXE5_KiLM...,None,https://www.francetvinfo.fr/replay-radio/c-est...
2565,Sept entreprises sur dix se disent touchées pa...,"Fri, 01 Mar 2024 20:34:44 +0100",[Franceinfo Avec Afp],economie,Les disparités sont nettes entre les secteurs....,https://www.francetvinfo.fr/pictures/f71kf5Ytm...,None,https://www.francetvinfo.fr/economie/entrepris...
2566,"Entreprise : après avoir frôlé la faillite, le...","Fri, 01 Mar 2024 18:57:01 +0100","[Camille Revel, Philippe Guillemot]",economie,"En ce moment, c'est la saison des résultats et...",https://www.francetvinfo.fr/pictures/eafiuZ9tK...,None,https://www.francetvinfo.fr/replay-radio/l-int...
2567,Ehpad : un grand groupe de maisons de retraite...,"Fri, 01 Mar 2024 16:02:50 +0100",[],economie,"Medicharme, un grand groupe de maisons de retr...",https://www.francetvinfo.fr/pictures/9b56oeGQL...,None,https://www.francetvinfo.fr/sante/senior/maltr...
